In [2430]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xgboost lightgbm catboost

In [2431]:
raw_df=pd.read_csv("Train_Data.csv")
test_df=pd.read_csv("Test_Data.csv")
cat_cols=['RIAGENDR','PAQ605','DIQ010']
num_cols=['BMXBMI','LBXGLU','LBXGLT','LBXIN','insulin_glucose_ratio','BMI_glucose_ratio']
new_features=[]
target_col=['age_group']

In [2434]:
# Feature engineering on raw_df
# raw_df["glucose_risk"] = ((raw_df["LBXGLU"] > 125) | (raw_df["LBXGLT"] > 140)).astype(int)
raw_df["insulin_glucose_ratio"] = raw_df["LBXIN"] / (raw_df["LBXGLU"] + 1)
# raw_df["high_bmi"] = (raw_df["BMXBMI"] > 30).astype(int)

# Same on test_df
# test_df["glucose_risk"] = ((test_df["LBXGLU"] > 125) | (test_df["LBXGLT"] > 140)).astype(int)
test_df["insulin_glucose_ratio"] = test_df["LBXIN"] / (test_df["LBXGLU"] + 1)
# test_df["high_bmi"] = (test_df["BMXBMI"] > 30).astype(int)
raw_df['BMI_glucose_ratio'] = raw_df['BMXBMI'] / (raw_df['LBXGLU'] + 1)
# raw_df['insulin_times_glucose'] = raw_df['LBXIN'] * raw_df['LBXGLU']
test_df['BMI_glucose_ratio'] = test_df['BMXBMI'] / (test_df['LBXGLU'] + 1)
# test_df['insulin_times_glucose'] = test_df['LBXIN'] * test_df['LBXGLU']

In [2436]:
raw_df = raw_df.dropna(subset=['age_group'])
raw_df = raw_df[raw_df['PAQ605'] != 7]
df=raw_df[cat_cols+num_cols+new_features]
test_df=test_df[cat_cols+num_cols+new_features]

In [2438]:
from sklearn.model_selection import train_test_split
target=raw_df['age_group']
train_data,val_data,train_y,val_y=train_test_split(df,target,test_size=0.2,random_state=42)

In [2440]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. Impute categorical separately
mode_imputer = SimpleImputer(strategy="most_frequent")
train_data[cat_cols] = mode_imputer.fit_transform(train_data[cat_cols])
val_data[cat_cols] = mode_imputer.transform(val_data[cat_cols])
test_df[cat_cols] = mode_imputer.transform(test_df[cat_cols])

# 2. Gender-wise KNN imputation function
def genderwise_knn_impute(df, num_cols, n_neighbors=3):
    df_imputed = df.copy()
    for gender in df['RIAGENDR'].unique():
        subset = df[df['RIAGENDR'] == gender]
        
        # Scale only this gender subset
        scaler = StandardScaler()
        scaled = scaler.fit_transform(subset[num_cols])
        
        # Apply KNN
        knn_imputer = KNNImputer(n_neighbors=n_neighbors)
        imputed = knn_imputer.fit_transform(scaled)
        
        # Inverse scale back to original values
        unscaled = scaler.inverse_transform(imputed)
        df_imputed.loc[subset.index, num_cols] = unscaled
        
    return df_imputed

# 3. Apply to all datasets
train_data = genderwise_knn_impute(train_data, num_cols)
val_data = genderwise_knn_impute(val_data, num_cols)
test_df = genderwise_knn_impute(test_df, num_cols)


In [2400]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=42)
# train_data, train_y = smote.fit_resample(train_data, train_y)

In [2402]:
# from sklearn.preprocessing import LabelEncoder
# encoder=LabelEncoder()
# train_y=encoder.fit_transform(train_y)
# val_y=encoder.transform(val_y)

In [2326]:
# train_data.isna().sum()

In [2404]:
# from sklearn.preprocessing import OneHotEncoder
# import pandas as pd
# cat_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# cat_encoder.fit(train_data[cat_cols])
# encoded_list=list(cat_encoder.get_feature_names_out(cat_cols))
# train_data[encoded_list]=cat_encoder.transform(train_data[cat_cols])
# val_data[encoded_list]=cat_encoder.transform(val_data[cat_cols])
# test_df[encoded_list]=cat_encoder.transform(test_df[cat_cols])

In [2406]:
# train_data = train_data.drop(columns=cat_cols)
# val_data = val_data.drop(columns=cat_cols)
# test_df = test_df.drop(columns=cat_cols)

In [2426]:
# from sklearn.preprocessing import StandardScaler

# scaler=StandardScaler()
# train_data=scaler.fit_transform(train_data)
# val_data=scaler.transform(val_data)
# test_df=scaler.transform(test_df)

In [2442]:
# from sklearn.svm import SVC
# from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.metrics import accuracy_score, classification_report


# xgb=XGBClassifier(
#     n_estimators=700,
#     learning_rate=0.02,
#     scale_pos_weight=6,
#     max_depth=7)
# xgb.fit(train_data,train_y)
# pred=xgb.predict(val_data)
# print(accuracy_score(val_y,pred))
# print(classification_report(val_y,pred))
# # rf=DecisionTreeClassifier(
# #     max_depth=8,
# #     random_state=42
# # )
# # rf.fit(train_data,train_y)
# # pred=rf.predict(val_data)
# # print(accuracy_score(val_y,pred))
# # print(classification_report(val_y,pred))
# # # # Define and train the model
# # lgbm = LGBMClassifier(
# #     n_estimators=500,
# #     learning_rate=0.05,
# #     max_depth=12,
# #     random_state=42
# # )

# # lgbm.fit(train_data, train_y)

# # # Make predictions
# # pred = lgbm.predict(val_data)

# # # Evaluate
# # print("Accuracy:", accuracy_score(val_y, pred))
# # print(classification_report(val_y, pred))

In [2444]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

# Ensure categorical columns are integers
for col in cat_cols + new_features:
    train_data[col] = train_data[col].astype(int)
    val_data[col] = val_data[col].astype(int)
    test_df[col]=test_df[col].astype(int)
categorical_indices=[0,1,2]
model = CatBoostClassifier(
        iterations=700,
        learning_rate=0.02,
        depth=7,
        class_weights=[3,10],
        cat_features=categorical_indices,
        verbose=0,
        l2_leaf_reg=3,
        random_state=42
    )
model.fit(train_data, train_y, eval_set=(val_data, val_y), use_best_model=True)
print(classification_report(val_y,model.predict(val_data)))
# precision    recall  f1-score   support

#        Adult       0.89      0.87      0.88       327
#       Senior       0.39      0.44      0.41        64

#     accuracy                           0.80       391
#    macro avg       0.64      0.65      0.64       391
# weighted avg       0.81      0.80      0.80       391
#   precision    recall  f1-score   support

#        Adult       0.90      0.85      0.87       327
#       Senior       0.40      0.52      0.45        64

#     accuracy                           0.80       391
#    macro avg       0.65      0.68      0.66       391
# weighted avg       0.82      0.80      0.81       391

              precision    recall  f1-score   support

       Adult       0.90      0.85      0.87       327
      Senior       0.40      0.52      0.45        64

    accuracy                           0.80       391
   macro avg       0.65      0.68      0.66       391
weighted avg       0.82      0.80      0.81       391



In [2224]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
top_features = [
    'LBXGLT',     # 19.77
    'BMXBMI',     # 18.56
    'insulin_glucose_ratio',  # 12.80
    'LBXGLU',     # 12.74
    'PAQ605',     # 10.91
    'LBXIN'       # 10.88
]

# Filter datasets to only top 6 features
train_top6 = train_data[top_features]
val_top6 = val_data[top_features]

# If any of these features are categorical, update their indices

# Define model
model_top6 = CatBoostClassifier(
        iterations=700,
        learning_rate=0.03,
        depth=6,
        class_weights=[3,10],
        verbose=0,
        random_state=42
)

# Train
model_top6.fit(train_top6, train_y, eval_set=(val_top6, val_y), use_best_model=True)

# Evaluate
preds = model_top6.predict(val_top6)
print(classification_report(val_y, preds))


              precision    recall  f1-score   support

       Adult       0.88      0.85      0.86       327
      Senior       0.34      0.41      0.37        64

    accuracy                           0.77       391
   macro avg       0.61      0.63      0.62       391
weighted avg       0.79      0.77      0.78       391



In [2446]:
# Get feature importances
importance = model.feature_importances_

# Get feature names (as list, assuming you're using a DataFrame as input)
feature_names = train_data.columns

# Combine into a DataFrame for easier viewing
importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

# Sort by importance (descending)
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Print
print(importances_df)


                 Feature  Importance
5                 LBXGLT   23.720917
4                 LBXGLU   12.319211
3                 BMXBMI   12.135905
1                 PAQ605   11.704549
8      BMI_glucose_ratio   11.217565
6                  LBXIN   10.774151
7  insulin_glucose_ratio   10.299347
2                 DIQ010    4.090423
0               RIAGENDR    3.737933


In [1482]:
# less_important = ['LBXGLU', 'PAQ605', 'DIQ010']  # double check column names

# train_data_reduced = train_data.drop(columns=less_important)
# val_data_reduced = val_data.drop(columns=less_important)

# model_reduced = CatBoostClassifier(
#     iterations=500,
#     learning_rate=0.01,
#     depth=5,
#     class_weights=[1,5],
#     cat_features=[train_data_reduced.columns.get_loc(col) for col in cat_cols if col in train_data_reduced.columns],
#     verbose=0,
#     random_strength=0.6,  # adds noise to avoid overfitting
#     feature_border_type='MinEntropy',
#     random_state=42
# )
# model_reduced.fit(train_data_reduced, train_y, eval_set=(val_data_reduced, val_y), use_best_model=True)

# from sklearn.metrics import classification_report
# print(classification_report(val_y, model_reduced.predict(val_data_reduced)))

In [2334]:
probs = model.predict_proba(test_df)


In [2336]:
import numpy as np

# Get probability for class 1 (Senior)
senior_probs = probs[:, 1]

# Count how many are close to 0.5
confused_count = np.sum((senior_probs > 0.45) & (senior_probs < 0.55))
total = len(senior_probs)
print(f"{confused_count}/{total} predictions are near 50% → potentially confused.")


25/312 predictions are near 50% → potentially confused.


In [2338]:
test_pred=model.predict(test_df)

In [2340]:
label_map = {"Adult": 0, "Senior": 1}
test_pred = pd.Series(test_pred).map(label_map).values

In [2342]:
submission = pd.DataFrame({
    "age_group": test_pred
})

In [2344]:
len(test_pred)

312

In [2346]:
submission.to_csv("submission3.csv", index=False)

In [2482]:
# Combine train and val
full_X = pd.concat([train_data, val_data], axis=0)
full_y = pd.concat([train_y, val_y], axis=0)

# Ensure same preprocessing (imputation, scaling, etc.)
# If already done before combining, skip this.

# Train the final model
final_model = CatBoostClassifier(
        iterations=400,
        learning_rate=0.05,
        depth=5,
        class_weights=[3,10],
        cat_features=categorical_indices,
        verbose=0,
        random_strength=0.5,
        l2_leaf_reg=3,
        random_state=42
)

final_model.fit(full_X, full_y)

# Final prediction
final_preds = final_model.predict(test_df)
final_preds = pd.Series(final_preds).map(label_map).values
# Optional: Save final predictions
submission = pd.DataFrame({
    "age_group": final_preds
})
submission.to_csv('final_submission.csv', index=False)


In [2483]:
print(classification_report(full_y, final_model.predict(full_X)))
print(model.predict_proba(full_X))

              precision    recall  f1-score   support

       Adult       0.97      0.93      0.95      1637
      Senior       0.71      0.86      0.78       314

    accuracy                           0.92      1951
   macro avg       0.84      0.90      0.87      1951
weighted avg       0.93      0.92      0.92      1951

[[0.78151791 0.21848209]
 [0.43243456 0.56756544]
 [0.86160717 0.13839283]
 ...
 [0.5603337  0.4396663 ]
 [0.73905983 0.26094017]
 [0.77525296 0.22474704]]
